## Quiz 2

In [28]:
import pandas as pd 
import numpy as np
import scipy
from scipy.stats import chi2_contingency
from statsmodels.stats.weightstats import zconfint

## Question 1

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [6]:
df = pd.read_csv('water.txt', sep='\t')

In [8]:
df.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [13]:
round(df[['mortality','hardness']].corr(),4)

,mortality,hardness
mortality,1.0000,-0.6548
hardness,-0.6548,1.0000


## Question 2

В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки. 

In [14]:
round(df[['mortality','hardness']].corr(method='spearman'),4)

,mortality,hardness
mortality,1.0000,-0.6317
hardness,-0.6317,1.0000


## Question 3

Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [15]:
round(df[df.location == 'South'][['mortality','hardness']].corr(),4)

,mortality,hardness
mortality,1.0000,-0.6022
hardness,-0.6022,1.0000


In [16]:
round(df[df.location != 'South'][['mortality','hardness']].corr(),4)

,mortality,hardness
mortality,1.0000,-0.3686
hardness,-0.3686,1.0000


## Question 4

Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.  

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

In [19]:
n_month_man = 239
n_month_woman = 203
n_rare_man = 515
n_rare_woman = 718

In [20]:
from math import sqrt
def mcc(a, b, c, d):

    x = (a + b) * (a + c) * (b + d) * (c + d)
    return ((a * d) - (b * c)) / sqrt(x)

In [21]:
round(mcc(n_rare_woman, n_month_woman, n_rare_man, n_month_man), 4)

0.109

## Question 5

В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

In [59]:
obs = np.array([[n_rare_woman, n_month_woman], [n_rare_man, n_month_man]])
chi2_contingency(obs)[1]

1.0558987006638725e-05

## Question 6

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

Z-критерий для разности долей (независимые выборки)

In [31]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [35]:
woman = np.concatenate((np.ones(n_month_woman), np.zeros(n_rare_woman)), axis=None)
man = np.concatenate((np.ones(n_month_man), np.zeros(n_rare_man)), axis=None)

In [40]:
proportions_diff_confint_ind(man, woman)

(0.053905233215813156, 0.13922183141523897)

## Question 7

Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили 5.5×10−85.5\times10^{-8}5.5×10−8, нужно ввести 8).

In [41]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [42]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [44]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(man, woman)))

p-value: 0.000008


## Question 8

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [45]:
happiness = np.array( [[197., 111., 33. ],
                       [382., 685., 331.],
                       [110., 342., 333.]] )

In [47]:
round(chi2_contingency(happiness)[0],4)

293.6831

##  Question 9
На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−85.5\times10^{-8}5.5×10−8, нужно ввести 8).

In [49]:
chi2_contingency(happiness)[1]

2.4964299580093467e-62

## Question 10
Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.

In [58]:
chi2 = chi2_contingency(happiness)[0]
n = happiness.sum()
round(np.sqrt(chi2 / (n*(min(happiness.shape)-1))), 4)

0.2412